In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# 1. Загрузка данных
data = pd.read_csv('processed_data.csv')  # путь к вашему файлу с данными

# 2. Предобработка текста: токенизация + удаление стоп-слов (для русского языка)
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    tokens = word_tokenize(str(text).lower())
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(filtered_tokens)

data['comment'] = data['comment'].astype(str).apply(preprocess_text)

# 3. Предобработка данных: приведение к нижнему регистру и очистка
categories = [
    'Вопрос решен',
    'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Нравится скорость отработки заявок',
    'Понравилось выполнение заявки',
    'Другое'
]

# Унификация текста: приведение к нижнему регистру и удаление лишних пробелов
data['comment'] = data['comment'].astype(str).str.lower().str.replace(r'\s+', ' ', regex=True).str.strip()

labels = data[categories].values.astype(int)

# 4. Разделение на обучающую и валидационную выборки с стратификацией по одному из классов (например, первому)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['comment'].values,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels.argmax(axis=1)  # баланс по первому классу
)

# 5. Токенизация и подготовка датасета с русской моделью BERT
model_name = 'DeepPavlov/rubert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

class CommentsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask= encoding['attention_mask'].squeeze(0)
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.FloatTensor(label)
        }

train_dataset = CommentsDataset(train_texts, train_labels, tokenizer)
val_dataset   = CommentsDataset(val_texts,   val_labels,   tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16)

# 6. Модель с несколькими выходами (multi-label)
class BertMultiLabelClassifier(nn.Module):
    def __init__(self, dropout=0.3):
        super(BertMultiLabelClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, len(categories))
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [batch_size, hidden_size]
        pooled_output = self.dropout(pooled_output)
        logits= self.classifier(pooled_output)  # [batch_size, num_classes]
        return logits

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model= BertMultiLabelClassifier()
model.to(device)

# 7. Обучение с планировщиком lr и клиппингом градиентов
criterion= nn.BCEWithLogitsLoss()
optimizer= optim.AdamW(model.parameters(), lr=2e-5)

num_epochs=4  # увеличьте число эпох для лучшего обучения

total_steps= len(train_loader)*num_epochs

scheduler= get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1*total_steps),
    num_training_steps=total_steps
)

for epoch in range(num_epochs):
    model.train()
    total_loss=0
    
    all_preds_train=[]
    all_labels_train=[]
    
    for batch in train_loader:
        input_ids= batch['input_ids'].to(device)
        attention_mask= batch['attention_mask'].to(device)
        labels= batch['labels'].to(device)

        optimizer.zero_grad()
        outputs= model(input_ids=input_ids, attention_mask=attention_mask)
        loss= criterion(outputs, labels)
        loss.backward()

        # градиентный клиппинг для стабилизации обучения
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        total_loss+= loss.item()

        # Для метрик на обучении (по желанию можно не считать каждую итерацию)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Игорь.А\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


Epoch 1/4, Loss: 0.5456
Epoch 2/4, Loss: 0.4082
Epoch 3/4, Loss: 0.3500
Epoch 4/4, Loss: 0.3221


In [ ]:
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score

# Переключение модели в режим оценки
model.eval()

all_preds = []
all_true = []

threshold = 0.5

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].cpu().numpy()  # преобразуем метки в numpy

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.sigmoid(outputs).cpu().numpy()

        all_preds.extend(preds)
        all_true.extend(labels)

# Преобразование списков в numpy массивы
true_labels = np.array(all_true)  # shape: (num_samples, num_classes)
preds_array = np.array(all_preds)

# Бинаризация предсказаний по порогу
pred_labels = preds_array >= threshold

# Вычисление F1-score (macro и micro)
f1_macro = f1_score(true_labels, pred_labels, average='macro')
f1_micro = f1_score(true_labels, pred_labels, average='micro')

print(f"F1-score (macro): {f1_macro:.4f}")
print(f"F1-score (micro): {f1_micro:.4f}")

# Вывод F1-score для каждого класса
for i, category in enumerate(categories):
    f1_class = f1_score(true_labels[:, i], pred_labels[:, i])
    print(f"F1-score для '{category}': {f1_class:.4f}")

print('--------------------------------------------------------------------------')

# Расчет ROC-AUC по каждому классу
roc_auc_scores = []
for i, category in enumerate(categories):
    try:
        score = roc_auc_score(true_labels[:, i], preds_array[:, i])
    except ValueError as e:
        score = None
        print(f"ROC-AUC для '{category}': недоступен ({e})")
    roc_auc_scores.append(score)

# Вывод ROC-AUC по классам
for i, category in enumerate(categories):
    score = roc_auc_scores[i]
    print(f"ROC-AUC для '{category}': {score if score is not None else 'недоступен'}")

# Средний ROC-AUC по классам
valid_scores = [score for score in roc_auc_scores if score is not None]
if valid_scores:
    roc_auc_mean = np.mean(valid_scores)
    print(f"Средний ROC-AUC по классам: {roc_auc_mean:.4f}")
else:
    print("Нет доступных значений ROC-AUC для вычисления среднего.")

F1-score (macro): 0.4503
F1-score (micro): 0.6731
F1-score для 'Вопрос решен': 0.3500
F1-score для 'Нравится качество выполнения заявки': 0.0000
F1-score для 'Нравится качество работы сотрудников': 0.8378
F1-score для 'Нравится скорость отработки заявок': 0.9163
F1-score для 'Понравилось выполнение заявки': 0.0000
F1-score для 'Другое': 0.5979
--------------------------------------------------------------------------
ROC-AUC для 'Вопрос решен': 0.7933189655172413
ROC-AUC для 'Нравится качество выполнения заявки': 0.799066742081448
ROC-AUC для 'Нравится качество работы сотрудников': 0.9405164092664092
ROC-AUC для 'Нравится скорость отработки заявок': 0.9598694316436253
ROC-AUC для 'Понравилось выполнение заявки': 0.48353853169174643
ROC-AUC для 'Другое': 0.8818519327364053
Средний ROC-AUC по классам: 0.8097
